# TensorFlow BYOM:
## カスタムトレーニングスクリプトでトレーニングし、Neoでコンパイルし、SageMakerで展開

このノートブックは、[TensorFlow MNIST distributed training notebook](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/tensorflow_distributed_mnist/tensorflow_distributed_mnist.ipynb) の拡張版で、Amazon SageMaker Neoを用いた例です。同じ分類タスクを実行しますが、今回はNeo APIバックエンドを使用して訓練モデルをコンパイルし、ハードウェアの選択に最適化します。 最後に、Neo Deep Learning Runtimeを使用してコンパイルされたモデルを使って、SageMakerにリアルタイムでホストされたエンドポイントを設定しています。

### 環境の設定

In [ ]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

### MNIST datasetのダウンロード

In [ ]:
import utils
from tensorflow.contrib.learn.python.learn.datasets import mnist
import tensorflow as tf

data_sets = mnist.read_data_sets('data', dtype=tf.uint8, reshape=False, validation_size=5000)

utils.convert_to(data_sets.train, 'train', 'data')
utils.convert_to(data_sets.validation, 'validation', 'data')
utils.convert_to(data_sets.test, 'test', 'data')

## データのアップロード
```sagemaker.Session.upload_data```関数を使ってデータセットをS3の場所にアップロードします。
戻り値の`inputs`は、トレーニングジョブを開始するときに使用します。

In [ ]:
inputs = sagemaker_session.upload_data(path='data', key_prefix='data/DEMO-mnist')

## 分散トレーニング用のスクリプトを作成する
エントリポイントとして使用するネットワークモデルのコードは次のとおりです。

In [ ]:
!pygmentize 'mnist.py'

このスクリプトは、[TensorFlow MNIST example](https://github.com/tensorflow/models/tree/master/official/mnist)です。 
これは、`` model_fn（features, labels, mode） ``を提供し、学習、評価、推論に使用されます。 トレーニングスクリプトの詳細については、[TensorFlow MNIST distributed training notebook](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker-python-sdk/tensorflow_distributed_mnist/tensorflow_distributed_mnist.ipynb) を参照してください。

学習用スクリプトの最後には、Neo Deep Learning Runtimeで使用する2つの追加機能があります。
* `neo_preprocess（payload、content_type）`：各受信リクエストのペイロードとContent-Typeを受け取り、NumPy配列を返す関数
* `neo_postprocess（result）`：Deep Learining Runtimeによって生成された予測結果を受け取り、応答本体を返す関数

## sagemaker.TensorFlow estimatorを使用してトレーニングジョブを作成する

In [ ]:
from sagemaker.tensorflow import TensorFlow

mnist_estimator = TensorFlow(entry_point='mnist.py',
                             role=role,
                             framework_version='1.11.0',
                             training_steps=1000, 
                             evaluation_steps=100,
                             train_instance_count=2,
                             train_instance_type='ml.c5.2xlarge')

%time mnist_estimator.fit(inputs)

**```fit```** メソッドは、2つの **ml.c5.2xlarge** インスタンスでトレーニングジョブを作成します。 上記のログには、学習、評価、**トレーニングステップ数の増分** などのインスタンスが表示されます。

学習の最後に、学習ジョブは、TF servingのために保存されたモデルを生成します。

### 訓練されたモデルを展開して予測を準備する（既存の手法）

deploy（）メソッドは、予測リクエストをリアルタイムで処理するエンドポイントを作成します。

In [ ]:
mnist_predictor = mnist_estimator.deploy(initial_instance_count=1,
                                         instance_type='ml.c5.4xlarge')

## エンドポイントの呼び出し

In [ ]:
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

for i in range(10):
    data = mnist.test.images[i].tolist()
    tensor_proto = tf.make_tensor_proto(values=np.asarray(data), shape=[1, len(data)], dtype=tf.float32)
    predict_response = mnist_predictor.predict(tensor_proto)
    
    print("========================================")
    label = np.argmax(mnist.test.labels[i])
    print("label is {}".format(label))
    prediction = predict_response['outputs']['classes']['int64_val'][0]
    print("prediction is {}".format(prediction))

## エンドポイントの削除

In [ ]:
sagemaker.Session().delete_endpoint(mnist_predictor.endpoint)

## Neoを使用して訓練されたモデルを展開する

これでモデルはNeoによってコンパイルされ、選択したハードウェアに最適化されました。 これを行うには `` TensorFlowEstimator.compile_model``メソッドを使用しています。 この例では、ターゲットハードウェアは `` 'ml_c5'``です。 必要に応じて、これらを他のサポートされているターゲットハードウェアに変更できます。

### モデルをコンパイルする
`` input_shape``はモデルの入力テンソルの定義で、 `` output_path``はコンパイルされたモデルがS3に格納される場所です。 **重要。 次のコマンドでパーミッションエラーが発生した場合は、スクロールして `get_execution_role（）`によって返された実行ロールの値を探します。 ロールは `` output_path``で指定されたS3バケットにアクセスする必要があります。**

In [ ]:
output_path = '/'.join(mnist_estimator.output_path.split('/')[:-1])
optimized_estimator = mnist_estimator.compile_model(target_instance_family='ml_c5', 
                              input_shape={'data':[1, 784]},  # Batch size 1, 3 channels, 224x224 Images.
                              output_path=output_path,
                              framework='tensorflow', framework_version='1.11.0')

## コンパイルされたモデルのデプロイ

In [ ]:
optimized_predictor = optimized_estimator.deploy(initial_instance_count = 1,
                                                 instance_type = 'ml.c5.4xlarge')

In [ ]:
# The neo_preprocess() function expects an image in the request body
# But the MNIST example data is saved as NumPy array.
# So we convert it to PNG before invoking the endpoint
def png_serializer(data):
    im = PIL.Image.fromarray(data.reshape((28,28))*255).convert('L')
    f = io.BytesIO()
    im.save(f, format='png')
    f.seek(0)
    return f.read()

optimized_predictor.content_type = 'application/x-image'
optimized_predictor.serializer = png_serializer

## エンドポイントの読み出し

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
from IPython import display
import PIL.Image
import io

mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

for i in range(10):
    data = mnist.test.images[i]
    # Display image
    im = PIL.Image.fromarray(data.reshape((28,28))*255).convert('L')
    display.display(im)
    # Invoke endpoint with image
    predict_response = optimized_predictor.predict(data)
    
    print("========================================")
    label = np.argmax(mnist.test.labels[i])
    print("label is {}".format(label))
    prediction = predict_response
    print("prediction is {}".format(prediction))

## エンドポイントの削除

In [ ]:
sagemaker.Session().delete_endpoint(optimized_predictor.endpoint)